# Rendu intermédaire du 6 octobre 2023 par Théo Duez

## Présentation du code

Importons dans un premier temps quelques packages qui nous sera utile :

In [ ]:
using Plots             # For plotting results
using LaTeXStrings      # For the utilisation of Latex in Plots
using Interpolations    # For interpolation used in the animation
using LinearAlgebra     # For matrix computation
using FFTW              # For Fourier computation

Voici maintenant le code qui permet de calculer la dynamique au cours du temps de l'équation de schödinger avec une méthode pseudospectrale en espace et de splitting en temps.

In [ ]:
function dynamics(ψ₀fun::Base.Callable = x->1/(2π)^(1/4)*exp(-x^2/4), Vfun::Base.Callable = (x,t) -> 0; L::Real = 20, Nₓ::Int = 1000, T::Real = 4, Nₜ::Int = 1000)

    # INPUTS
    #   - L     : Length difining the spatial domain [-L/2, L/2] on wich we perform the simulation,
    #   - T     : Duration defining the temporal domain [0,T] on wich we perform the simulation,
    #   - ψ₀fun : Initial condition at time 0,
    #   - Vfun  : Potential in the spatial domain,
    #   - Nₓ    : Number of points used for an uniform discretization of the space,
    #   - Nₜ     : Number of points used for an uniform discretization of the time.

    # OUTPUTS
    #   - matrix of dimension Nₓ×(Nₜ+1) the columns of which correspond to the solution at each time beginning at the initial state.

    # Space discretization
    space = range(-L/2,L/2,Nₓ+1)[1:Nₓ]

    # Initial condition
    ψ₀ = ψ₀fun.(space)
    
    # Timestep
    Δt = eltype(ψ₀)(T/Nₜ)

    # Time discretization
    time = range(Δt,T,Nₜ)
    
    # Potential
    V = hcat([Vfun.(space,tᵢ) for tᵢ in time]...)

    # Matrix to reach next each step
    stepΔ = exp.(-im/2*eltype(ψ₀)(0.5)*(fftfreq(Nₓ,Nₓ)*2*π/L).^2*Δt)
    stepV = exp.(-im*V*Δt)

    # Matrix to store the solution
    sol = zeros(Complex, Nₓ,Nₜ+1)
    sol[:,1]=ψ₀
    
    # Loop to compute the solution for each time step
    for i in 2:Nₜ+1
        sol[:,i] = ifft(stepΔ .* fft(stepV[i] .* sol[:,i-1]))
    end
    sol
end

Voici maintenant le code correspondant pour l'animation du résultat :

In [ ]:
function animation(ψ::Matrix; d::Real = 10, fps::Int = 30, L::Real = 10)

    # INPUTS
    #   - ψ     : Solution Matrix given by the dynamics function,
    #   - d     : Duration wanted for the animation,
    #   - fps   : Number of frame per secondes,
    #   - L     : Width of the spatial domain (only used for dispay),

    # OUTPUTS
    #   - An animation through the time of the real, imaginary and module part of the wave function ψ.
    
    # Some assertions to check validity of parameters
    @assert L > 0
    @assert d > 0
    @assert fps > 0
    
    # set the min and maximum values of the plot, to scale the axis
    m = min(0, min(real.(ψ)...), min(imag.(ψ)...))
    M = max(abs.(ψ)...)

    # Total frame
    nbframe = fps * d

    # Space and time vector
    Nₓ,Nₜ = size(ψ)
    time = range(0,1,Nₜ)
    space = range(-L/2,L/2,Nₓ+1)[1:Nₓ]

    anim = @animate for n in 1:nbframe

        t = n / (fps*d)

        linear_interpolation(time, ψ[1,:])
        linear_interpolation(time, ψ[1,:])(t)

        interp_linear = [linear_interpolation(time, ψ[i,:])(t) for i in eachindex(ψ[:,1])]
        
        pltreal = plot(space,real.(interp_linear), label = "Real", size=(1200,400), xlim = [-L/2,L/2], ylim= [m,M], legend=:bottomleft, lc=:green)
        pltimag = plot(space,imag.(interp_linear), label = "Imag", size=(1200,400), xlim = [-L/2,L/2], ylim= [m,M], legend=:bottomleft, lc=:red)
        pltabs  = plot(space,abs.(interp_linear), label = "Abs",  size=(1200,400), xlim = [-L/2,L/2], ylim= [m,M], legend=:bottomleft, lc=:black)

        plot(pltreal, pltimag, pltabs, layout=(1,3), plot_title="Approximation by the Fourier Transform")

    end

    gif(anim, fps = fps)
end

## Exemples

Montrons quelques exemples. Créeons pour cela une fonction qui générera nos conditions initiales.


In [ ]:
function gaussian(μ::Real = 0.0, σ::Real = 1.0, p₀::Real = 0.0)
    x-> 1/(2π*σ^2)^(1/4)*exp(-(x-μ)^2/(4*σ^2))*exp(im*p₀*x)
end


### Potentiel nul avec condition initiale gaussienne immobile

In [ ]:
# Paramètres de simulation
T = 50
L = 10
Nₜ = 100
Nₓ = 100

# Potential
V = (x,t) -> 0

# Intial Condition
ψ₀ = gaussian(0.0, 1.0, 0.0)

# Computation of the dynamics
sol = dynamics(ψ₀, V, L = L, Nₓ=Nₓ,T=T,Nₜ=Nₜ)

# Animation
animation(sol; L = L)

### Potentiel nul avec condition initiale gaussienne mobile

In [ ]:
# Paramètres de simulation
T = 50
L = 10
Nₜ = 100
Nₓ = 100

# Potential
V = (x,t) -> 0

# Intial Condition
ψ₀ = gaussian(0.0, 1.0, 1.0)

# Computation of the dynamics
sol = dynamics(ψ₀, V, L = L, Nₓ=Nₓ,T=T,Nₜ=Nₜ)

# Animation
animation(sol; L = L)

### Barièrre de potentiel avec condition initiale gaussienne immobile

In [ ]:
# Paramètres de simulation
T = 10
L = 50
Nₜ = 1000
Nₓ = 1000

# Potential
V = (x,t) -> abs(x)>L/4 ? 10.0 : 0.0

# Intial Condition
ψ₀ = gaussian(0.0, 1.0, 0.0)

# Computation of the dynamics
sol = dynamics(ψ₀, V, L = L, Nₓ=Nₓ,T=T,Nₜ=Nₜ)

# Animation
animation(sol; L = L, d = 20)

On observe bien une onde evanescente qui se propage au delà de la barière de potentiel. Cependant, la simualtion présente des phases d'évolution rapide et des phases d'évolution lente. Il se pourrait que le schéma numérique soit instable pour ce type de potentiel.

## Compléxité (Partie non finie)

La complexité de la transformée de fourier rapide et de son inverse est O(Nₓlog(Nₓ)). Les multiplications points par points sont en O(Nₓ) donc une complexité à l'intérieur de la boucle en O(Nₓlog(Nₓ)). Comme nous effectuons ces opérations à chaque pas de temps, la complexité gobale est O(NₜNₓlog(Nₓ)), les autres opérations hors de la boucle ayant une complexité en O(NₓNₜ) (due à la création de la matrice V) négligeable. 

Vérifions cela dans un premier temps t(Nₓ,Nₜ) pour la complexité spatiale en mesurant le temps que prend l'algorithme pour différentes valeurs de Nₓ et à une valeur de Nₜ fixée, en traçant la fonction $Nₓ \mapsto t(Nₓ,Nₜ)/Nₓ\log(Nₓ)$.

In [ ]:
ComplexiteSpace = Float64[]
VectNₓ = 10:1:1001
Nₜ = 1
M = 100
t₁ = @elapsed (dynamics(Nₓ=2, Nₜ=Nₜ))
for Nₓ in VectNₓ
    t = 0
    for _ in 1:M
        t+= @elapsed (dynamics(Nₓ=Nₓ, Nₜ=Nₜ))
    end
    push!(ComplexiteSpace, t/(M*t₁))
end

plot(VectNₓ[2:end],ComplexiteSpace[2:end])
plot!(VectNₓ[2:end], VectNₓ[2:end].* log.(VectNₓ[2:end])*t₁)

## Conservation de la densité de probabilité

Nous pouvons vérfier que notre solution $ψ$ est de norme L₂ en espace au carré constante égale à 1 au cours du temps. En fait, nous allons plutôt vérifier $\frac{||ψ(t)||_2}{||ψ₀||_2} - 1= 0$ pour temps $t\in[0,T]$ car du fait de la troncature à l'intervale $[-L/2,L/2]$, la norme L₂ de la solution initiale peut être inférieur à 1, comme c'est le cas pour la gaussienne.

In [ ]:
# Paramètres de simulation
T = 50
L = 10
Nₜ = 1000
Nₓ = 1000

# Potential
V = (x,t) -> 0

# Intial Condition
ψ₀ = gaussian()

# Computation of the dynamics
sol = dynamics(ψ₀, V, L = L, Nₓ=Nₓ,T=T,Nₜ=Nₜ)

norm₂ψ₀ = norm(sol[:,1])
plot(range(0,T,Nₜ), [norm(sol[:,i]) / norm₂ψ₀ for i in 1:Nₜ].-1, label = L"\frac{||ψ(t)||_2}{||ψ_0||_2}-1")
xlabel!(L"T")

On remarque que étant donné, l'échelle en abscisse que l'amplitude de probablité $||ψ(t)||^2_2$ est conservé au cours du temps. Cependant l'erreur commise par le schéma suit clairement une tendance qui pourrait être à très long termes un problème. Cela est due à la transformée de Fourier et son inverse qui commette une erreur de par l'approximation finie des séries de fourier employées.

## Comparaison avec une solution analytique (en cours)

On peut également comparé la solution numérique avec la solution quasi-exacte connue dans le cas d'une condition initiale gaussienne et un champ de potentiel nul.

In [ ]:
# Paramètres de simulation
T = 10
L = 10
Nₜ = 100
Nₓ = 100

#
V = (x,t) -> 0
ψ₀(x) = 1/(sqrt(2π))*exp(-x^2)

sol = dynamics(ψ₀, V, L = L, Nₓ=Nₓ,T=T,Nₜ=Nₜ)


α(t) = im*(2π/L)^2*t + 1/4
C(t) = sqrt(α(t))/(sqrt(π))
ψₜᵣᵤₑ(x,t) = C(t)*exp(-x^2/(4*α(t)))

animation(sol; L = L)